In [54]:
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
data = pd.read_csv('./HW2 附件/HW2_hr-analytics_train.csv')

In [18]:
df = data.copy()
# check nan
def check_nan(df):
    return df.isna().any().any()

def onehot_encoding(df, column_name):
    df_encoded = pd.get_dummies(df, columns=[column_name], prefix=[column_name])
    print(df.shape, "->", df_encoded.shape)
    return df_encoded

def pprint(output = '\n', show_time = False): # print and fprint at the same time
    global filename
    print(output)
    with open(filename, 'a') as f:
        if show_time:
            f.write(datetime.now().strftime("[%Y-%m-%d %H:%M:%S] "))

        f.write(str(output))
        f.write('\n')
        
def check_non_numeric_values(df, column_name):
    """
    Check if any value in the specified column is not int or float.

    Parameters:
    df (DataFrame): The DataFrame to check.
    column_name (str): The name of the column to check.

    Returns:
    bool: True if any non-numeric value is found, False otherwise.
    """
    # Get the specified column
    column = df[column_name]

    # Check if any value is not int or float
    non_numeric_values = column[~column.apply(lambda x: isinstance(x, (int, float)))]
    
    return not non_numeric_values.empty

def count_unique_values(df, column_name):
    """
    Count the number of unique values in the specified column.

    Parameters:
    df (DataFrame): The DataFrame to check.
    column_name (str): The name of the column to count unique values for.

    Returns:
    int: The count of unique values.
    """
    unique_values = df[column_name].nunique()
    return unique_values

col_not_num = []
for col in df.columns:
    if check_non_numeric_values(df, col):
        col_not_num.append(col)

print(f"Doing onehot encoding on col: {col_not_num}")
for col in col_not_num:
    df = onehot_encoding(df, col)


Doing onehot encoding on col: ['sales', 'salary']
(10000, 10) -> (10000, 19)
(10000, 19) -> (10000, 21)


In [32]:
target_column = 'left'
test_size = 0.2
random_state = 42
X = df.drop(columns=[target_column])
y = df[target_column]

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=test_size, random_state=random_state)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=random_state)

In [79]:
# Method 4: Feature Selection
from sklearn.feature_selection import SelectKBest, f_classif

# Use SelectKBest to select the top k features based on ANOVA F-statistics
k = 1  # Set the number of features to select
selector = SelectKBest(score_func=f_classif, k=k)
X_train_selected = selector.fit_transform(X_train, y_train)
X_val_selected = selector.transform(X_val)

# Create a new logistic regression model
logistic_regression = LogisticRegression()

# Fit the model with selected features
logistic_regression.fit(X_train_selected, y_train)

selector = SelectKBest(score_func=f_classif, k=k)
y_pred = logistic_regression.predict(X_val_selected)
accuracy = accuracy_score(y_val, y_pred)
print("Improved Accuracy:", accuracy)

# Method 5: Class Imbalance Handling
from imblearn.over_sampling import SMOTE

# Use SMOTE to oversample the minority class
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Train a logistic regression model on the resampled data
logistic_regression.fit(X_resampled, y_resampled)
logistic_regression.fit(X_train, y_train)
logistic_regression.fit(X_train, y_train)


# Evaluate the model on the test set
y_pred = logistic_regression.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Improved Accuracy:", accuracy)

# Method 6:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Create polynomial features
degree = 5  # Choose the degree of polynomial features
polyreg = make_pipeline(PolynomialFeatures(degree), LogisticRegression())
polyreg.fit(X_train, y_train)
y_pred = polyreg.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Improved Accuracy:", accuracy)

Improved Accuracy: 0.763
Improved Accuracy: 0.776


/Users/liushiwen/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/liushiwen/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

In [57]:
# TRAIN

model = LogisticRegression()
model.fit(X_train, y_train)


# Define hyperparameters to search
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Regularization strength
    'penalty': ['l1', 'l2'],  # Regularization type
    'solver': ['liblinear'],  # Suitable for small datasets
}

# Create a logistic regression model
logistic_regression = LogisticRegression()

# Perform hyperparameter tuning with cross-validation
grid_search = GridSearchCV(logistic_regression, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Evaluate the model on the test set
y_pred = best_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Improved Accuracy:", accuracy)


/Users/liushiwen/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Improved Accuracy: 0.782


In [56]:


y_pred = model.predict(X_val)
print(y_pred.shape)
# Calculate the confusion matrix
confusion = confusion_matrix(y_val, y_pred)
print(confusion)

def metrics(y_test, y_pred):
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

    # Calculate precision
    precision = precision_score(y_test, y_pred)
    print("Precision:", precision)

    # Calculate recall (sensitivity)
    recall = recall_score(y_test, y_pred)
    print("Recall (Sensitivity):", recall)

    # Calculate F1 score (harmonic mean of precision and recall)
    f1 = f1_score(y_test, y_pred)
    print("F1 Score:", f1)
    # Calculate ROC AUC (Receiver Operating Characteristic Area Under the Curve)
    roc_auc = roc_auc_score(y_test, y_pred)
    print("ROC AUC:", roc_auc)

metrics(y_test, y_pred)


(1000,)
[[700  59]
 [165  76]]
Accuracy: 0.68
Precision: 0.2074074074074074
Recall (Sensitivity): 0.11618257261410789
F1 Score: 0.14893617021276595
ROC AUC: 0.48760380277609217


In [48]:
result = y_pred.reshape(-1, 1)  # Reshape to (1000, 1)
# Create a DataFrame with one column 'left'
result_df = pd.DataFrame(data=result, columns=['left'])
print(result_df)
result_df.to_csv('HW2_hr-analytics_test_sol.csv', index=False)

     left
0       0
1       0
2       1
3       0
4       0
..    ...
995     0
996     0
997     0
998     0
999     0

[1000 rows x 1 columns]
